In [125]:
path="/content/drive/My Drive/project data/"

In [126]:
import pandas as pd
import numpy as np

In [127]:
train=pd.read_csv(path+"train.csv")
data=pd.read_csv(path+"preprocessed_data.csv")

In [128]:
labels=train.iloc[:,2:]
labels.head(10)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [129]:
classes=list(labels.columns)
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [130]:
data.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,vector
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,[ 0.00593334 0.07003178 -0.2012667 0.377534...
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,[-0.2508106 0.01673899 0.31538136 -0.121969...
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,[ 0.41935966 -0.12742983 -0.48621123 0.245375...
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,[ 0.32448252 -0.40889367 -0.00564199 0.136562...
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,[-0.04981409 0.68221519 -0.62567751 0.101100...


In [131]:
def vec(string):
    string = string.replace('\n', '')
    string = string.replace('[ ', '')
    string = string.replace('[', '')
    string = string.replace(']', '')
    lst = string.split()
    for i in range(len(lst)):
        lst[i] = float(lst[i])
    return lst

In [132]:
lst = []
for ele in data['vector']:
    lst.append(vec(ele))

In [133]:
data['vector'] = lst
data.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,vector
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[0.00593334, 0.07003178, -0.2012667, 0.3775345..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[-0.2508106, 0.01673899, 0.31538136, -0.121969..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[0.41935966, -0.12742983, -0.48621123, 0.24537..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[0.32448252, -0.40889367, -0.00564199, 0.13656..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[-0.04981409, 0.68221519, -0.62567751, 0.10110..."


In [134]:
temp = []
for ele in data['vector']:
    temp.append(np.array(ele))
data['vector'] = temp

In [135]:
y=data.iloc[:,3:9]
x=data['vector']
print(x.shape)
print(y.shape)

(159571,)
(159571, 6)


In [136]:
from sklearn.model_selection import train_test_split
X_train, X_val_test, y_train, y_val_test = train_test_split(x,y,test_size=0.3,random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_val_test,y_val_test,test_size=0.5,random_state=1)

In [137]:
x_tr = []
for ele in X_train:
  x_tr.append(ele)
x_tr=np.array(x_tr)

x_va = []
for ele in X_val:
  x_va.append(ele)
x_va=np.array(x_va)

x_ts = []
for ele in X_test:
  x_ts.append(ele)
x_ts=np.array(x_ts)

In [138]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model=LogisticRegression(C=20.0, max_iter=1000000)

In [ ]:
## Binary Relevance

In [139]:
scores_roc_aoc = []

for label_name in classes:
    print('Class:', label_name)

    model.fit(x_tr, y_train[label_name])
    preds_train = model.predict(x_tr)
    train_roc_auc = roc_auc_score(y_train[label_name], preds_train)
    print('Train ROC AUC Score:', train_roc_auc)

    preds_val = model.predict(x_va)
    val_roc_auc = roc_auc_score(y_val[label_name], preds_val)
    print('Val ROC AUC Score:', val_roc_auc)

    preds_test = model.predict(x_ts)
    test_roc_auc = roc_auc_score(y_test[label_name], preds_test)
    print('Test ROC AUC Score:', test_roc_auc)
    scores_roc_aoc.append(test_roc_auc)

    print()


print(np.mean(scores_roc_aoc))

Class: toxic
Train ROC AUC Score: 0.7518182318302802
Val ROC AUC Score: 0.7482186594356357
Test ROC AUC Score: 0.7487698632025699

Class: severe_toxic
Train ROC AUC Score: 0.6090461648160052
Val ROC AUC Score: 0.6132522135672442
Test ROC AUC Score: 0.584763030032673

Class: obscene
Train ROC AUC Score: 0.7510319025085368
Val ROC AUC Score: 0.7518540501066314
Test ROC AUC Score: 0.753925370199633

Class: threat
Train ROC AUC Score: 0.5507777777777778
Val ROC AUC Score: 0.5607060417218829
Test ROC AUC Score: 0.530956572769953

Class: insult
Train ROC AUC Score: 0.7009275324603561
Val ROC AUC Score: 0.6990594030872183
Test ROC AUC Score: 0.7033423254112624

Class: identity_hate
Train ROC AUC Score: 0.5447198152176701
Val ROC AUC Score: 0.5558101646960942
Test ROC AUC Score: 0.5462924044776464

0.6446749276822896


In [ ]:
## Classifier chains

In [140]:
from scipy.sparse import hstack, csr_matrix

def add_feat(x, feat):
  return hstack([x, csr_matrix(feat).T], 'csr')

In [141]:
scores_roc_aoc = []

for label_name in classes:
    print('Class:', label_name)

    model.fit(x_tr, y_train[label_name])
    preds_train = model.predict(x_tr)
    train_roc_auc = roc_auc_score(y_train[label_name], preds_train)
    print('Train ROC AUC Score:', train_roc_auc)

    preds_val = model.predict(x_va)
    val_roc_auc = roc_auc_score(y_val[label_name], preds_val)
    print('Val ROC AUC Score:', val_roc_auc)

    preds_test = model.predict(x_ts)
    test_roc_auc = roc_auc_score(y_test[label_name], preds_test)
    print('Test ROC AUC Score:', test_roc_auc)
    scores_roc_aoc.append(test_roc_auc)

    X_train_feat = add_feat(x_tr, y_train[label_name])
    X_val_feat = add_feat(x_va, y_val[label_name])
    X_test_feat = add_feat(x_ts, y_test[label_name])
    print(X_train_feat.shape)
    print(X_val_feat.shape)
    print(X_test_feat.shape)
    print()

print(np.mean(scores_roc_aoc))

Class: toxic
Train ROC AUC Score: 0.7518182318302802
Val ROC AUC Score: 0.7482186594356357
Test ROC AUC Score: 0.7487698632025699
(111699, 101)
(23936, 101)
(23936, 101)

Class: severe_toxic
Train ROC AUC Score: 0.6090461648160052
Val ROC AUC Score: 0.6132522135672442
Test ROC AUC Score: 0.584763030032673
(111699, 101)
(23936, 101)
(23936, 101)

Class: obscene
Train ROC AUC Score: 0.7510319025085368
Val ROC AUC Score: 0.7518540501066314
Test ROC AUC Score: 0.753925370199633
(111699, 101)
(23936, 101)
(23936, 101)

Class: threat
Train ROC AUC Score: 0.5507777777777778
Val ROC AUC Score: 0.5607060417218829
Test ROC AUC Score: 0.530956572769953
(111699, 101)
(23936, 101)
(23936, 101)

Class: insult
Train ROC AUC Score: 0.7009275324603561
Val ROC AUC Score: 0.6990594030872183
Test ROC AUC Score: 0.7033423254112624
(111699, 101)
(23936, 101)
(23936, 101)

Class: identity_hate
Train ROC AUC Score: 0.5447198152176701
Val ROC AUC Score: 0.5558101646960942
Test ROC AUC Score: 0.5462924044776464